In [1]:
# Dependencies
import pymongo
# import datetime
import pandas as pd
from bson.json_util import dumps

from religion_cleaning1 import relig_dict

# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.senate_db

# Declare the collections
senators = db.senators_db
uspop = db.us_db
religion = db.religion_db

{'Baptist Family (ET)': 9.94, 'Methodist Family (ET)': 0.25, 'Non-denominational Family (ET)': 4.61, 'Lutheran Family (ET)': 1.92, 'Presbyterian (ET)': 0.73, 'Pentecostal Family (ET)': 3.27, 'Episcopalian/Anglican Family (ET)': 0.0, 'Restorationist Family (ET)': 1.57, 'Congregationalist Family (ET)': 0.04, 'Holiness Family (ET)': 0.78, 'Reformed Family (ET)': 0.06, 'Adventist Family (ET)': 0.49, 'Anabaptist Family (ET)': 0.2, 'Pietist Family (ET)': 0.1, 'Other Evangelical': 0.14, 'NonSpecific Protestant Family (ET)': 1.51, 'Baptist Family (MT)': 2.55, 'Methodist Family (MT)': 4.27, 'Non-Denominational Family (MT)': 1.06, 'Lutheran Family (MT)': 2.71, 'Presbyterian Family (MT)': 1.33, 'Episcopalian/Anglican Family (MT)': 1.41, 'Restorationist Family (MT)': 0.18, 'Congregationalist Family (MT)': 0.61, 'Reformed Family (MT)': 0.06, 'Anabaptist Family (MT)': 0.06, 'Friends Family (MT)': 0.02, 'NonSpecific Protestant Family (MT)': 1.92, 'Baptist Family (HBPT)': 3.59, 'Methodist Family (HBPT

In [2]:
sen_path = '../resources/SPopulation.csv'
sen_data = pd.read_csv(sen_path)

us_path = '../resources/CensusAttributesData.csv'
us_data = pd.read_csv(us_path)

religion_path = '../resources/ReligiousAffiliationByState.csv'
relig_data = pd.read_csv(religion_path)

In [3]:

senators.delete_many({})

sen_list = []
for p in sen_data.iterrows():
    row = p[1] # p[0] is the index of the row
        
    sen_row = {
        'st': row['st'],
        'state': row['State'],
        'senator': row['Senator'],
        'party': row['party'],
        'born': row['Born'],
        'assumed_office': row['assumed_office'],
        'term_up': row['assumed_office'],
        'population': row['population'],
        'gender': row['gender'],
        'race': row['race'],
        'religion': row['Religion']
    }
    
    sen_list.append(sen_row)
    senators.insert_one(sen_row)

In [4]:
us_data.columns

Index(['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'NAME', 'SEX', 'ORIGIN',
       'RACE', 'AGE', 'CENSUS2010POP', 'ESTIMATESBASE2010', 'POPESTIMATE2010',
       'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013',
       'POPESTIMATE2014', 'POPESTIMATE2015', 'POPESTIMATE2016',
       'POPESTIMATE2017', 'POPESTIMATE2018', 'POPESTIMATE2019'],
      dtype='object')

In [5]:
cleaned_us_data = us_data[['STATE', 'NAME', 'SEX', 'ORIGIN', 'RACE', 'AGE', 'POPESTIMATE2019']]
cleaned_us_data.head()

,STATE,NAME,SEX,ORIGIN,RACE,AGE,POPESTIMATE2019
0,1,Alabama,0,0,1,0,35594
1,1,Alabama,0,0,1,1,36679
2,1,Alabama,0,0,1,2,37904
3,1,Alabama,0,0,1,3,38510
4,1,Alabama,0,0,1,4,38476


In [6]:
#uspop.delete_many({})


us_list = []
for p in cleaned_us_data.iterrows():
    row = p[1] # p[0] is the index of the row

    if p[0] % 10000 == 0:
        print(p[0]/10000)
        
    us_row = {
        'NAME': row['NAME'],
        'STATE': row['STATE'],
        'SEX': row['SEX'],
        'ORIGIN': row['ORIGIN'],
        'RACE': row['RACE'],
        'AGE': row['AGE'],
        'POPESTIMATE2019': row['POPESTIMATE2019']
    }

    
    
    us_list.append(us_row)
    #uspop.insert_one(us_row)

# WHY DOES THIS TAKE SO LONG
# 46 secs without mongo
# 3 minutes with??

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0


In [8]:
wyoming = []
for b in us_list:
    if b['NAME'] == 'Wyoming':
        wyoming.append(b)

In [9]:
religion.delete_many({})


# relig_list = []
# for p in relig_dict:
    # print(p)
    # row = p[1] # p[0] is the index of the row


    # #relig_cols = ['index', 'state', 'catholic', 'jewish', 'muslim', 'buddhist', 'hindu', 'agnostic']
        
    # relig_row = {
    #     'State': row['state'],
    #     'Catholic': row['catholic'],
    #     'Jewish': row['jewish'],
    #     'Muslim': row['muslim'],
    #     'Buddhist': row['buddhist'],
    #     'Hindu': row['hindu'],
    #     'Agnostic': row['agnostic']
    # }

    
    
    # relig_list.append(p)
religion.insert_one(relig_dict)


#print(type(relig_list))
    # should definitely make this into a function sigh

In [10]:
json_sen_data = dumps(list(senators.find()))
 
with open('senators.json', 'w') as file: 
    file.write(json_sen_data) 

In [11]:
json_us_data = dumps(list(uspop.find()))
 
with open('uspop.json', 'w') as file: 
    file.write(json_us_data) 

In [12]:
json_relig_data = dumps(list(religion.find()))
 
with open('religion.json', 'w') as file: 
    file.write(json_relig_data) 